# Gowalla - Modelling
After looking at the data in the EDA, it's time to build our prediction model. The main goal is to predict if two users will be friends based on their behavior and their place in the network.

## Initialization

In [ ]:
#Imports
import pandas as pd
import networkx as nx
import math
import random
import numpy as np

from haversine import haversine_vector

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import RandomizedSearchCV

In [2]:
random.seed(42)

pd.set_option('display.float_format', '{:.10f}'.format)

In [3]:
edges_path = 'data/Gowalla_edges.txt'

# Data loading
G = nx.read_edgelist(edges_path, nodetype=int)
checkins_df = pd.read_parquet('data/EDA_output/checkins.parquet')

## Feature Engineering

Initially, a simple random 80/20 split of the edges was considered for creating the training and test sets. However, this approach was identified as methodologically flawed for a temporal dataset. A random split could create a scenario where the model is trained on check-in features from 2010 to predict a link that was held out for the test set, even if that link represents a friendship from 2009. This constitutes a form of data leakage, where information from the future is used to predict the past, leading to an overly optimistic and unrealistic evaluation of the model's performance."

To avoid this temporal leakage and address the limitations of the data, we reframe the prediction task. Given the absence of friendship formation timestamps that makes true temporal prediction impossible, we *shift from predicting link formation to predicting link existence within the final network snapshot*.

> Can we use a user's early activity to effectively distinguish between user pairs who are friends and those who are not in the final, complete social network?

In this new objective we accept **structural leakage** by using the final graph for embeddings, but we explicitly define the task around this by predicting the *final state*, not the *formation*.

### Feature Dataset Train/Test Split

We proceed to make the 80/20 split of the check-ins. We have two alternative:
- Split by volume of check-ins (e.g. train set will have 80% of check-ins)
- Split by time passed (e.g train set will have check-ins that appears in the first 80% of time)

In [4]:
# Split the check-ins dataframe by time
min_ts = checkins_df['check-in_datetime'].min()
max_ts = checkins_df['check-in_datetime'].max()

time_diff = max_ts - min_ts

time_cutoff = min_ts + pd.to_timedelta(0.8 * time_diff)

t_train_df = checkins_df[checkins_df['check-in_datetime'] < time_cutoff]
t_test_df = checkins_df[checkins_df['check-in_datetime'] >= time_cutoff]

print("Check-ins cutoff at: ", time_cutoff)
print(f"Train set size: {len(t_train_df)} - Corresponding percentage of volume: {len(t_train_df)/len(checkins_df)*100:.1f}")
print(f"Test set size:  {len(t_test_df)} - Corresponding percentage of volume: {len(t_test_df)/len(checkins_df)*100:.1f}")

Check-ins cutoff at:  2010-06-20 00:33:12.400000+00:00
Train set size: 2939173 - Corresponding percentage of volume: 45.6
Test set size:  3503690 - Corresponding percentage of volume: 54.4


In [5]:
# Split the check-ins dataframe by volume
volume_cutoff = math.ceil(len(checkins_df) * 0.80)

checkins_df = checkins_df.sort_values(by=['check-in_datetime'])

v_train_df = checkins_df.iloc[:volume_cutoff]
v_test_df = checkins_df.iloc[volume_cutoff:]

volume_cutoff_ts = checkins_df.iloc[volume_cutoff]['check-in_datetime']

print("Check-ins cutoff at: ", volume_cutoff_ts)
print(f"Train set size: {len(v_train_df)} - Corresponding percentage of time: {((volume_cutoff_ts - min_ts) / time_diff) * 100:.1f}")
print(f"Test set size:  {len(v_test_df)} - Corresponding percentage of time: {((max_ts - volume_cutoff_ts) / time_diff) * 100:.1f}")

Check-ins cutoff at:  2010-09-14 14:24:41+00:00
Train set size: 5154291 - Corresponding percentage of time: 93.8
Test set size:  1288572 - Corresponding percentage of time: 6.2


Either way we obtain an overall imbalanced split, and either options have pros and cons:
- Option "Time": This is the most honest representation of a real-world temporal prediction task, but the features built from this sparser data might be weak.
- Option "Volume": The features built will be much stronger and the model will surely perform better, but the separation between past and future is tiny.

As the goal of the notebook is learning, we proceed with the time-based split.

### Negative Sampling

To create a balanced dataset we take an equal number of positive and negative samples. To make the training set challenging, we use a mixed strategy:
- 70% of negative samples are "hard" negatives generated with random walks with a path distance of 2.
- 20% are "medium" negatives generated with random walks with a path distance of 4, representing users who are further apart but still connected.
- 10% are "easy" random negatives, to ensure the model learns the global structure of the graph.

In [6]:
# Negative Sampling Generation Functions
def _random_walk_hard_negative(graph, count, walk_distance=2, max_attempts=100):
    """Generates a batch of hard negative samples."""
    if walk_distance < 1:
        raise ValueError("walk_distance must be >= 1")

    edges = list(graph.edges())
    if not edges:
        return []

    generated_samples = []
    for _ in range(count):
        for _ in range(max_attempts):
            u, current_node = random.choice(edges)

            # Make hops
            for _ in range(walk_distance - 1):
                neighbors = list(graph.neighbors(current_node))
                if not neighbors:
                    current_node = None
                    break
                current_node = random.choice(neighbors)

            if current_node is None:
                continue

            v = current_node

            # Validate edge and add to the batch
            if v != u and not graph.has_edge(u, v):
                generated_samples.append((u, v))
                break

    return generated_samples

def _random_negative_sampling(graph, count, max_attempts=100):
    """Generates a batch of random negative samples."""
    nodes = list(graph.nodes())
    if len(nodes) < 2:
        return []

    generated_samples = []
    for _ in range(count):
        for _ in range(max_attempts):
            u, v = random.sample(nodes, 2)
            if not graph.has_edge(u, v):
                generated_samples.append((u, v))
                break

    return generated_samples

In [7]:
# Main Negative Sampling Function to combine different methods
def generate_negative_samples_set(
    graph,
    sampling_functions,
    sampling_weights,
    ratio=1.0,
    seed=None
):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    if len(sampling_functions) != len(sampling_weights):
        raise ValueError("sampling_functions and sampling_weights must be of the same length")

    total_neg_samples = int(len(graph.edges) * ratio)

    weights = np.array(sampling_weights, dtype=float)
    weights /= weights.sum()
    samples_for_each_function = np.random.multinomial(total_neg_samples, weights)

    negative_samples = set()

    for func, count in zip(sampling_functions, samples_for_each_function):
        if count == 0:
            continue

        new_samples = func(graph, count)
        negative_samples.update(new_samples)

    return negative_samples

In [8]:
# Generate Negative Samples
positive_samples = list(G.edges)

sampling_functions = [
    lambda g, c: _random_negative_sampling(g, c),
    lambda g, c: _random_walk_hard_negative(g, c, walk_distance=4),
    lambda g, c: _random_walk_hard_negative(g, c, walk_distance=2)
]

sampling_weights = [0.1, 0.2, 0.7]

negative_samples = generate_negative_samples_set(
    graph=G,
    sampling_functions=sampling_functions,
    sampling_weights=sampling_weights,
    ratio=1.1, # Allow to compensate duplicates
    seed=42
)

print(f"Generated {len(negative_samples)} ({len(negative_samples)/len(positive_samples)*100:.2f}%) negative samples.")

Generated 973408 (102.43%) negative samples.


In [9]:
# Truncate the exceeding part
negative_samples = random.sample(list(negative_samples), len(positive_samples))

In [10]:
# Combine and create labels
combined_samples = positive_samples + negative_samples
samples_labels = [1] * len(positive_samples) + [0] * len(negative_samples)

### Node Embeddings

The goal of **Node Embedding** is to learn a *dense vector representation* for each user that captures their neighborhood and role in the network. This approach allows the model to automatically learn features from the graph topology, serving as a powerful alternative to manual feature engineering with standard measures (e.g., Common Neighbors).

We chose the **Node2Vec** algorithm for this task. An initial test with a standard Python implementation of Node2Vec proved to be extremely slow. The first phase of the algorithm, *"computing transition probabilities"* was single-threaded and was projected to take several hours. A Stack Overflow discussion<sup>[[1][1]]</sup> highlighted this common bottleneck and pointed towards more performant, specialized libraries.

[1]: https://stackoverflow.com/questions/60276191/is-there-any-way-to-make-node2vec-faster

Based on this research, we switched to **GRAPE**<sup>[[2][2]]</sup>, a fast graph processing and embedding library. GRAPE is written in *Rust* and *Python*, developed primarily at the University of Milan, and is designed for speed and scalability on massive graphs. It parallelizes the entire Node2Vec process, including the pre-computation step, which dramatically reduces the training time.  
While the library's documentation was found to be in a raw state, we were able to successfully implement it for our purposes.

[2]: https://github.com/AnacletoLAB/grape

To make our workflow efficient and reproducible, we performed this computationally expensive step in a separate script (`train_embeddings.py`) and saved the resulting embeddings to a file. The model was configured with a neutral baseline set of hyperparameters:
- `embedding_size` (dimensions): 128
- `walk_length` (length of each walk): 80
- `iterations` (number of walks per node): 10
- `return_weight` ($p = 1 / \text{return\_weight}$): 1.0
- `explore_weight` ($q = 1 / \text{explore\_weight}$): 1.0

In the Node2Vec framework `return_weight` and `explore_weight` set to `1.0` corresponds to the parameters `p` and `q`, which removes any search bias and makes the algorithm equivalent to the **DeepWalk** algorithm.

The training process, which ran over *30 epochs*, required *35 minutes* to complete.

In [11]:
# Load Embeddings
embedding_path = "embeddings/gowalla_node2vec_embeddings.parquet"
embedding_df = pd.read_parquet(embedding_path)
embedding_df.index = embedding_df.index.astype(int)

In [12]:
# Show result
embedding_df.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-70.2993240356,104.7642517090,214.4183197021,32.5812606812,45.5297737122,-224.9113006592,-34.7072448730,-298.1494750977,129.5424499512,67.4690933228,...,-105.4124908447,-51.9473838806,-102.9604339600,-141.7391204834,204.9244232178,-261.5464477539,-77.5561447144,19.7141284943,23.8973484039,195.5611419678
1,-260.2064819336,-104.6907653809,31.3309593201,119.9266281128,-124.3474426270,135.4678497314,0.4026212692,-39.9399147034,61.0377693176,151.5090789795,...,325.2360534668,-9.6202507019,19.4639568329,-81.2915039062,26.7115669250,-47.5153923035,-77.6566009521,-122.7394409180,87.3364486694,-115.6673278809
2,-91.6686706543,-46.4467735291,86.5811386108,-220.1943511963,12.7583608627,8.1809272766,-22.7226924896,-86.8817749023,-82.5780715942,-254.7404327393,...,158.5266723633,258.4648742676,-213.0023193359,-65.7916030884,264.3765258789,80.9072494507,-95.8889389038,-58.6533508301,-153.2696228027,-241.1257171631
3,73.5213851929,-146.0536651611,-68.6905746460,224.7186889648,-89.2506484985,-123.0056457520,-43.8369445801,-157.7464294434,-73.6201477051,-41.8115425110,...,238.8144989014,-32.8614349365,121.9555130005,230.7531433105,91.0168304443,-223.9833679199,-84.6682815552,125.3443145752,-201.8149261475,-279.3994750977
4,-0.6959481239,1.2965914011,-2.1391603947,0.3263847530,1.1603585482,0.7213216424,0.4505686462,-1.4355301857,1.0976371765,0.3337603509,...,-1.6340924501,-0.7793192863,2.8571474552,0.3815542758,1.8547649384,-1.0311043262,2.7614219189,0.5874806643,0.7654225230,-1.9500918388


In [13]:
print(f'DataFrame with {embedding_df.shape[0]} rows and {embedding_df.shape[1]} columns')

DataFrame with 196591 rows and 128 columns


The next step is to create a single feature vector for each *pair* of users in our sample set. This vector needs to describe the relationship between the two users. The **Hadamard Product** is a simple yet effective method that should capture the interaction and agreement between two users embedding along each dimension.

In [14]:
# Efficient vectorized Hadamard Product
samples_df = pd.DataFrame(combined_samples, columns=['source', 'destination'])
source_vectors = embedding_df.loc[(samples_df['source'])].values
destination_vectors = embedding_df.loc[samples_df['destination']].values
hadamard_product = source_vectors * destination_vectors

In [15]:
# Create Pair Features dataframe with multi-index
pair_features_df = pd.DataFrame(
    hadamard_product,
    index=pd.MultiIndex.from_frame(samples_df),
    columns=[f'embed_hadamard_{i}' for i in range(hadamard_product.shape[1])]
)

In [16]:
# Show result
pair_features_df.head()

embed_hadamard_0  embed_hadamard_1  embed_hadamard_2  \
source destination                                                         
0      1            18292.3398437500 -10967.8496093750   6717.9316406250   
       2             6444.2456054688  -4865.9614257812  18564.5820312500   
       3            -5168.5039062500 -15301.2031250000 -14728.5175781250   
       4               48.9246826172    135.8364257812   -458.6751708984   
       5            -5373.4765625000  21647.4687500000  -9167.5957031250   

                    embed_hadamard_3  embed_hadamard_4  embed_hadamard_5  \
source destination                                                         
0      1             3907.3608398438  -5661.5107421875 -30468.2500000000   
       2            -7174.2094726562    580.8852539062  -1839.9830322266   
       3             7321.6181640625  -4063.5617675781  27665.3593750000   
       4               10.6340265274     52.8308639526   -162.2333831787   
       5            -2164.2231445312  -8469.9921875000  24528.3496093750   

                    embed_hadamard_6  embed_hadamard_7  embed_hadamard_8  \
source destination                                                         
0      1              -13.9738750458  11908.0644531250   7906.9819335938   
       2              788.6420288086  25903.7558593750 -10697.3652343750   
       3             1521.4595947266  47032.0156250000  -9536.9345703125   
       4              -15.6379966736    428.0025634766    142.1906127930   
       5            -5406.4482421875 -14682.1591796875   3649.0617675781   

                    embed_hadamard_9  ...  embed_hadamard_118  \
source destination                    ...                       
0      1            10222.1806640625  ...   -34283.9414062500   
       2           -17187.1054687500  ...   -16710.6914062500   
       3            -2820.9868164062  ...   -25174.0312500000   
       4               22.5185089111  ...      172.2537536621   
       5            -7673.9218750000  ...     4982.6293945312   

                    embed_hadamard_119  embed_hadamard_120  \
source destination                                           
0      1                499.7468566895    -2004.0174560547   
       2             -13426.5742187500    21930.8105468750   
       3               1707.0655517578   -12556.5927734375   
       4                 40.4835968018     -294.1731567383   
       5             -13844.1279296875    -2367.4938964844   

                    embed_hadamard_121  embed_hadamard_122  \
source destination                                           
0      1              11522.1865234375     5473.8525390625   
       2               9325.2441406250    54177.2070312500   
       3             -32706.7480468750    18651.5722656250   
       4                -54.0811691284      380.0866394043   
       5              16342.4062500000    -1967.7547607422   

                    embed_hadamard_123  embed_hadamard_124  \
source destination                                           
0      1              12427.4824218750     6022.7465820312   
       2             -21161.0039062500     7436.7763671875   
       3              58582.0546875000     6566.5454101562   
       4                269.6816711426     -214.1652374268   
       5              14392.5039062500   -11204.3603515625   

                    embed_hadamard_125  embed_hadamard_126  embed_hadamard_127  
source destination                                                              
0      1              -2419.7011718750     2087.1096191406   -22620.0351562500  
       2              -1156.2996826172    -3662.7375488281   -47154.8203125000  
       3               2471.0539550781    -4822.8417968750   -54639.6796875000  
       4                 11.5816688538       18.2915687561     -381.3621826172  
       5               2485.1079101562    -2291.1472167969   -32421.9785156250  

[5 rows x 128 columns]

In [17]:
# Free-up memory
del embedding_df
del samples_df
del source_vectors
del destination_vectors
del hadamard_product

### Domain Features

Here is a palette of candidate features that were considered for the link prediction model:
- **Radius of Gyration**: A measure of a user's typical travel radius, distinguishing "stayers" (small radius) from "travelers" (large radius).
- **Total Check-in Count**: The total number of check-ins for a user, indicating their overall activity level.
- **Unique Locations Count**: The number of distinct locations a user has visited, indicating their tendency to explore.
- **Jaccard Similarity of Visited Locations**: The overlap in the set of unique locations visited by two users, measuring shared lifestyle and interests.
- **Co-check-in Count**: The number of times two users checked into the same location within a short time window (e.g., 1 hour), indicating possible real-world interaction.
- **Haversine Distance between User Centroids**: The geographic distance between the average location (centroid) of all check-ins for each user.
- **Haversine Distance between Inferred Home Locations**: The geographic distance between the inferred "home" location of each user.
- **Explicit Topological Features**: Common Neighbors, Jaccard Coefficient, Adamic-Adar, Resource Allocation, Preferential Attachment

The first selection will be based on covering each domain of interaction with one feature that is thought to be the best:
- **Radius of Gyration**: Analyzed in the EDA, represent the travelers vs. stayers concept and is an indicator of heterophily.
- **Haversine Distance between Inferred Home Location**: the core finding in the EDA, represent the geographic proximity and has already been proved a strong indicator of homophily.
- **Jaccard Similarity of Visited Locations**: we didn't explicitly explore this in the EDA, but could strongly represent the shared habits for a pair of users.

In [18]:
checkins_per_user = checkins_df['user'].value_counts()
print(f'[Original Dataset] - Number of users with at least one check-in: {len(checkins_per_user)} ({len(checkins_per_user)/G.number_of_nodes()*100:.2f}%)')
checkins_per_user = t_train_df['user'].value_counts()
print(f'[Time Train Dataset] - Number of users with at least one check-in: {len(checkins_per_user)} ({len(checkins_per_user)/G.number_of_nodes()*100:.2f}%)')
checkins_per_user = v_train_df['user'].value_counts()
print(f'[Volume Train Dataset] - Number of users with at least one check-in: {len(checkins_per_user)} ({len(checkins_per_user)/G.number_of_nodes()*100:.2f}%)')


[Original Dataset] - Number of users with at least one check-in: 107092 (54.47%)
[Time Train Dataset] - Number of users with at least one check-in: 63526 (32.31%)
[Volume Train Dataset] - Number of users with at least one check-in: 94019 (47.82%)


#### Pair-Wise Radius of Gyration

We need to recalculate the Radius of Gyration using only the check-ins before the cutoff date.

In [19]:
# Calculate Radius of Gyration with only check-ins before cutoff
# Calculate Mean Centroid
users_mean_centroids = t_train_df.groupby('user').agg(
    mean_centroid_latitude=('latitude', 'mean'),
    mean_centroid_longitude=('longitude', 'mean')
)

# Merge Mean Centroid with check-ins dataframe
checkins_with_centroids = pd.merge(t_train_df, users_mean_centroids, on='user')
checkins_with_centroids = checkins_with_centroids[['user', 'latitude', 'longitude', 'mean_centroid_latitude', 'mean_centroid_longitude']]

# Zip to obtain coordinates pairs
checkins_coords = list(zip(checkins_with_centroids['latitude'], checkins_with_centroids['longitude']))
centroid_coords = list(zip(checkins_with_centroids['mean_centroid_latitude'], checkins_with_centroids['mean_centroid_longitude']))

# Efficient vectorized Haversine Distance
distances = haversine_vector(checkins_coords, centroid_coords)
checkins_with_centroids['sq_distance_from_centroid'] = distances**2

# Calculate Radius of Gyration and save in a new dataframe
radius_of_gyration = checkins_with_centroids.groupby('user')['sq_distance_from_centroid'].mean().apply(np.sqrt).rename('radius_of_gyration_km')
radius_of_gyration.index = radius_of_gyration.index.astype(int)

In [20]:
# Show results
radius_of_gyration.head()

user
0    1203.8236124683
4     421.4152289762
5      49.8019317812
9       5.5957428171
10      8.4755866231
Name: radius_of_gyration_km, dtype: float64

In [21]:
# Calculate the pair-wise feature: Radius of Gyration Absolute Difference
samples_df = pd.DataFrame(combined_samples, columns=['source', 'destination'])
source_rg_vector = samples_df['source'].map(radius_of_gyration)
destination_rg_vector = samples_df['destination'].map(radius_of_gyration)
abs_diff_rg = (source_rg_vector - destination_rg_vector).abs()

rg_feature_df = pd.DataFrame({
    'rg_source': source_rg_vector,
    'rg_destination': destination_rg_vector,
    'rg_abs_diff': abs_diff_rg
})
rg_feature_df.index = pd.MultiIndex.from_frame(samples_df)

In [22]:
# Check for NaN values
print(rg_feature_df.isna().sum(), "\n")

rg_na = rg_feature_df.isna().sum().loc['rg_abs_diff']
print(f"Radius of Gyration absolute difference missing for {rg_na} ({rg_na/len(combined_samples)*100:.4}%) pairs.")

rg_source          991583
rg_destination    1110422
rg_abs_diff       1470012
dtype: int64 

Radius of Gyration absolute difference missing for 1470012 (77.34%) pairs.


The **Radius of Gyration (RoG)** was calculated for all users based on their activity before the time-based cutoff date. As expected, and as a direct consequence of the network's growth, only 32.3% of the total users had check-in data within this period.

We now need to deal with the presence of numerous missing values due to having many users not active on the platform in this time range. To treat this information as a predictive signal rather than a data problem, we engineered a dedicated categorical feature, `activity_status`, to explicitly describe the activity state of each pair (`Both Active`, `Source Active Only`, etc.). This allows the model to learn distinct patterns for each scenario. With a feature that gives context the the RoG values, all the NaN can now be filled with zeros. 

In [23]:
# Fill the NaN values
rg_feature_df['rg_source'] = rg_feature_df['rg_source'].fillna(0)
rg_feature_df['rg_destination'] = rg_feature_df['rg_destination'].fillna(0)

rg_feature_df['rg_abs_diff'] = (rg_feature_df['rg_source'] - rg_feature_df['rg_destination']).abs()

In [24]:
# Add categorical feature for context
source_is_active = samples_df['source'].isin(radius_of_gyration.index)
destination_is_active = samples_df['destination'].isin(radius_of_gyration.index)

conditions = [
    source_is_active & destination_is_active,
    source_is_active & ~destination_is_active,
    ~source_is_active & destination_is_active,
    ~source_is_active & ~destination_is_active
]
choices = ['both_have', 'source_only', 'dest_only', 'neither_have']

activity_status = np.select(conditions, choices, default='Error')

rg_feature_df['activity_status'] = activity_status
rg_feature_df.head()

rg_source  rg_destination     rg_abs_diff  \
source destination                                                   
0      1           1203.8236124683    0.0000000000 1203.8236124683   
       2           1203.8236124683    0.0000000000 1203.8236124683   
       3           1203.8236124683    0.0000000000 1203.8236124683   
       4           1203.8236124683  421.4152289762  782.4083834921   
       5           1203.8236124683   49.8019317812 1154.0216806871   

                   activity_status  
source destination                  
0      1               source_only  
       2               source_only  
       3               source_only  
       4                 both_have  
       5                 both_have

In [25]:
# One-hot encoding categorical feature
rg_feature_df = pd.get_dummies(rg_feature_df, columns=['activity_status'], prefix='rg_status')
rg_feature_df.head()

rg_source  rg_destination     rg_abs_diff  \
source destination                                                   
0      1           1203.8236124683    0.0000000000 1203.8236124683   
       2           1203.8236124683    0.0000000000 1203.8236124683   
       3           1203.8236124683    0.0000000000 1203.8236124683   
       4           1203.8236124683  421.4152289762  782.4083834921   
       5           1203.8236124683   49.8019317812 1154.0216806871   

                    rg_status_both_have  rg_status_dest_only  \
source destination                                             
0      1                          False                False   
       2                          False                False   
       3                          False                False   
       4                           True                False   
       5                           True                False   

                    rg_status_neither_have  rg_status_source_only  
source destination                                                 
0      1                             False                   True  
       2                             False                   True  
       3                             False                   True  
       4                             False                  False  
       5                             False                  False

In [26]:
# Add results to the complete pair feature dataframe
pair_features_df = pair_features_df.join(rg_feature_df.drop(columns=['rg_source', 'rg_destination']))

In [27]:
# Free-Up Memory
del users_mean_centroids
del checkins_with_centroids
del checkins_coords
del centroid_coords
del distances
del radius_of_gyration
del samples_df
del source_rg_vector
del destination_rg_vector
del abs_diff_rg
del rg_feature_df
del source_is_active
del destination_is_active
del activity_status
del conditions

#### Haversine Distance between Inferred Home Locations

Again, all the calculations needs to be redone using only the check-ins before the cutoff date.

In [28]:
# Reuse code from EDA
inferred_home_df = t_train_df.groupby('user').agg(
    median_centroid_latitude=('latitude', 'median'),
    median_centroid_longitude=('longitude', 'median')
)

# Heuristic to infer Home Locations
home_hours = (t_train_df['check-in_datetime'].dt.hour >= 21) | (t_train_df['check-in_datetime'].dt.hour < 7)

# The grid will have 25x25 Kilometers cells
lat_step, lon_step = 0.25, 0.25
t_train_df['lat_bin'] = (t_train_df['latitude'] / lat_step).astype(int)
t_train_df['lon_bin'] = (t_train_df['longitude'] / lon_step).astype(int)

# Find the most visited grid cell for each user during home hours
home_cells = t_train_df[home_hours]\
    .groupby(['user', 'lat_bin', 'lon_bin'])\
    .size()\
    .reset_index(name='count')\
    .sort_values('count', ascending=False)\
    .drop_duplicates(subset='user')\
    .set_index('user')\
    [['lat_bin', 'lon_bin']]\
    .rename(columns={'lat_bin': 'home_lat_bin', 'lon_bin': 'home_lon_bin'})

# Chain operations to calculate the centroid of check-ins within each user's home cell
home_cell_centroids = (
    t_train_df.join(home_cells, on='user')
    .dropna(subset=['home_lat_bin'])
    .query('lat_bin == home_lat_bin and lon_bin == home_lon_bin')
    .groupby('user')
    .agg(
        home_cell_centroid_lat=('latitude', 'median'),
        home_cell_centroid_lon=('longitude', 'median')
    )
)

inferred_home_df = inferred_home_df.join(home_cell_centroids)
inferred_home_df.index = inferred_home_df.index.astype(int)
inferred_home_df = inferred_home_df[['home_cell_centroid_lat', 'home_cell_centroid_lon']]

In [29]:
inferred_home_df.isna().sum()

home_cell_centroid_lat    10739
home_cell_centroid_lon    10739
dtype: int64

In [30]:
samples_df = pd.DataFrame(combined_samples, columns=["source", "destination"])

samples_df = samples_df.join(inferred_home_df, on="source")
samples_df = samples_df.rename(columns={
    "home_cell_centroid_lat": "source_home_lat",
    "home_cell_centroid_lon": "source_home_lon",
})

samples_df = samples_df.join(inferred_home_df, on="destination")
samples_df = samples_df.rename(columns={
    "home_cell_centroid_lat": "dest_home_lat",
    "home_cell_centroid_lon": "dest_home_lon",
})

In [31]:
samples_df.index = pd.MultiIndex.from_frame(samples_df[["source", "destination"]])
samples_df = samples_df.drop(columns=["source", "destination"])

In [32]:
source_home_coords = list(zip(samples_df['source_home_lat'], samples_df['source_home_lon']))
dest_home_coords = list(zip(samples_df['dest_home_lat'], samples_df['dest_home_lon']))

distances = haversine_vector(source_home_coords, dest_home_coords)
samples_df['home_distance_km'] = distances

In [33]:
# Check for NaN values
print(samples_df.isna().sum(), "\n")

home_dist_na = samples_df.isna().sum().loc['home_distance_km']
print(f"Home Distance missing for {home_dist_na} ({home_dist_na/len(combined_samples)*100:.4}%) pairs.")

source_home_lat     1060573
source_home_lon     1060573
dest_home_lat       1201301
dest_home_lon       1201301
home_distance_km    1539456
dtype: int64 

Home Distance missing for 1539456 (81.0%) pairs.


In [34]:
source_has_home = samples_df['source_home_lat'].notna()
dest_has_home = samples_df['dest_home_lat'].notna()

conditions = [
    source_has_home & dest_has_home,
    source_has_home & ~dest_has_home,
    ~source_has_home & dest_has_home,
    ~source_has_home & ~dest_has_home
]

choices = ['both_have', 'source_only', 'dest_only', 'neither_have']
samples_df['home_status'] = np.select(conditions, choices, default='unknown')

The `home_distance_km` column now contains the calculated distances for pairs where both users had an inferred home, and NaN for all other pairs. A simple imputation with zero would be misleading, as it would imply that users with no inferred home live at the same location. Therefore, a more neutral and statistically robust approach is chosen: the missing distance values are imputed using the **median** of all the calculated distances.

In [35]:
# Impute the missing home distance values with the median of the distances
# TODO: could also try to impute with a very large distance
median_distance = samples_df['home_distance_km'].median()
samples_df['home_distance_km'] = samples_df['home_distance_km'].fillna(median_distance)

print("Median Home Distance: ", median_distance)

Median Home Distance:  832.7161963358135


In [36]:
# One-hot encoding categorical feature
samples_df = pd.get_dummies(samples_df, columns=['home_status'], prefix='home_status')

In [37]:
# Add results to the complete pair feature dataframe
pair_features_df = pair_features_df.join(samples_df.drop(columns=['source_home_lat', 'source_home_lon', 'dest_home_lat', 'dest_home_lon']))

In [38]:
# Free-Up Memory
del inferred_home_df
del home_cells
del home_hours
del home_cell_centroids
del samples_df
del source_home_coords
del dest_home_coords

#### Jaccard Similarity of Visited Locations

This is a new introduced feature that we didn't considered in the EDA phase. The goal here is to measure shared lifestyle and interests. We hypothesize that users who frequent the same set of locations, are more likely to be friends. The Jaccard Similarity of their visited location sets provides an intuitive measure of this behavioral overlap.

The similarity alone can be a weak signal if it's based on few shared location, but could be a strong one if it's based on many shared locations.
The initial idea was to weight the similarity with the visited location count, but having a nonlinear model like GBM we can give the model the raw components and let it discover the complex interactions itself.

In [39]:
# Create a dictionary of users - visited locations
user_location_sets = t_train_df.groupby('user')['location_id'].apply(set)
user_location_dict = user_location_sets.to_dict()

In [40]:
# Define the Location Overlap function
def _location_overlap(set1, set2):
    intersection_size = len(set1.intersection(set2))
    union_size = len(set1.union(set2))

    if union_size == 0:
        return 0.0, intersection_size, union_size

    return (intersection_size / union_size), intersection_size, union_size

In [41]:
# Calculate Jaccard Similarity using list comprehensions
samples_df = pd.DataFrame(combined_samples, columns=["source", "destination"])

locations_overlap_data = [
    _location_overlap(user_location_dict.get(source, set()), user_location_dict.get(destination, set()))
    for source, destination in zip(samples_df['source'], samples_df['destination'])
]

jaccard, intersect_sizes, union_sizes = zip(*locations_overlap_data)

jaccard_feature_df = pd.DataFrame({
    'jaccard_similarity_locations': jaccard,
    'intersect_locations': intersect_sizes,
    'unions_locations': union_sizes
})

jaccard_feature_df.index = pd.MultiIndex.from_frame(samples_df)

jaccard_feature_df.head()

jaccard_similarity_locations  intersect_locations  \
source destination                                                      
0      1                            0.0000000000                    0   
       2                            0.0000000000                    0   
       3                            0.0000000000                    0   
       4                            0.0294117647                    3   
       5                            0.0000000000                    0   

                    unions_locations  
source destination                    
0      1                          34  
       2                          34  
       3                          34  
       4                         102  
       5                          57

In [42]:
# Check for NaNs
jaccard_feature_df.isna().sum()

jaccard_similarity_locations    0
intersect_locations             0
unions_locations                0
dtype: int64

In [43]:
# Overview of the results
sim_not_zero = len(jaccard_feature_df[jaccard_feature_df['jaccard_similarity_locations'] > 0])
sim_zero = len(jaccard_feature_df[jaccard_feature_df['jaccard_similarity_locations'] == 0])

print(f"There are {sim_not_zero} ({sim_not_zero/len(jaccard_feature_df)*100:.2f}%) pairs with locations similarity >0.")
print(f"There are {sim_zero} ({sim_zero/len(jaccard_feature_df)*100:.2f}%) pairs with locations similarity =0.")

jaccard_feature_df.describe()

There are 110675 (5.82%) pairs with locations similarity >0.
There are 1789979 (94.18%) pairs with locations similarity =0.


,jaccard_similarity_locations,intersect_locations,unions_locations
count,1900654.0000000000,1900654.0000000000,1900654.0000000000
mean,0.0019025140,0.2376182093,56.9806692854
std,0.0141596191,2.1730818325,125.3736290873
min,0.0000000000,0.0000000000,0.0000000000
25%,0.0000000000,0.0000000000,0.0000000000
50%,0.0000000000,0.0000000000,14.0000000000
75%,0.0000000000,0.0000000000,60.0000000000
max,1.0000000000,445.0000000000,2988.0000000000


As was to be expected the vast majority of pairs don't have similar visited location, because the data is sparse from all over the world.

In [44]:
# Add results to the complete pair feature dataframe
pair_features_df = pair_features_df.join(jaccard_feature_df)

In [45]:
# Free-Up Memory
del jaccard_feature_df
del locations_overlap_data
del samples_df
del user_location_sets
del user_location_dict
del jaccard, intersect_sizes, union_sizes

### Save Results

In [46]:
pair_features_df.shape

(1900654, 141)

In [47]:
pair_features_df['is_friend'] = samples_labels
pair_features_df.to_parquet("data/engineered_features/final_modeling_dataset.parquet")

## Model Training

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna
import lightgbm as lgbm

In [2]:
modeling_df = pd.read_parquet("data/engineered_features/final_modeling_dataset.parquet")

In [3]:
modeling_df.shape

(1900654, 142)

In [4]:
# Final Train/Test Split
X = modeling_df.drop(columns=['is_friend'])
y = modeling_df['is_friend']

# Perform the 80/20 split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape:  {X_test.shape}")


X_train shape: (1520523, 141)
X_test shape:  (380131, 141)


In [5]:
del modeling_df, X, y

In [6]:
# Subsample Data for Hyperparameter Tuning
X_train_sample, _, y_train_sample, _ = train_test_split(
    X_train,
    y_train,
    train_size=0.1, # 10%
    random_state=42,
    stratify=y_train
)

In [ ]:
lgbm_dataset_sample = lgbm.Dataset(X_train_sample, label=y_train_sample)

def objective(trial):
    params = {
        # Forest params
        'n_estimators': trial.suggest_int('n_estimators', 200, 1000),

        # Tree params
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'num_leaves': trial.suggest_int('num_leaves', 8, 4096),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9), # bagging fraction
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9), # feature subset fraction

        # Model settings
        'objective': 'binary',
        'metric': 'auc',
        'random_state': 42,
        'n_jobs': -1,
        'verbose': 1
    }


    cv_results = lgbm.cv(
        params=params,
        train_set=lgbm_dataset_sample,
        num_boost_round=params['n_estimators'],
        nfold=5,
        stratified=True,
        seed=42,
        callbacks=[lgbm.early_stopping(10, verbose=True)]
    )

    best_score = max(cv_results['valid auc-mean']) # type: ignore

    return best_score


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True, gc_after_trial=True)

best_params = study.best_params

In [14]:
study.trials[0].values[0]

0.8759919906092769

In [15]:
best_score = study.best_value
tolerance = 0.01

candidates = [
    trial for trial in study.trials
    if trial.state == optuna.trial.TrialState.COMPLETE and
       trial.values[0] >= best_score - tolerance
]

min(candidates, key=lambda t: t.params["n_estimators"])

FrozenTrial(number=15, state=1, values=[0.8750329709173551], datetime_start=datetime.datetime(2025, 7, 13, 13, 19, 42, 240357), datetime_complete=datetime.datetime(2025, 7, 13, 13, 22, 7, 194358), params={'n_estimators': 375, 'max_depth': 11, 'num_leaves': 1133, 'learning_rate': 0.03877486168812111, 'subsample': 0.8460268736474088, 'colsample_bytree': 0.6915663877398335}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=1000, log=False, low=200, step=1), 'max_depth': IntDistribution(high=12, log=False, low=3, step=1), 'num_leaves': IntDistribution(high=4096, log=False, low=8, step=1), 'learning_rate': FloatDistribution(high=0.3, log=True, low=0.01, step=None), 'subsample': FloatDistribution(high=0.9, log=False, low=0.7, step=None), 'colsample_bytree': FloatDistribution(high=0.9, log=False, low=0.6, step=None)}, trial_id=15, value=None)